## Downloading and processing ERA5 data

In this tutorial, we will use the DLWP data module to fetch and pre-process data from ERA5 to use in a DLWP weather prediction model. For the sake of simplicity, we use only a select few variables over a few years.

#### Python packages required here not in the base requirements

Let's start by installing the `cdsapi` package, which is required for retrieval of data. (See the README for packages already required for DLWP that need to also be installed.) Note that to use `cdsapi` you will need to register for an API key at CDS, following [their instructions](https://cds.climate.copernicus.eu/api-how-to).

In [1]:
%conda install -c conda-forge cdsapi

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: - ^C
/ 
Note: you may need to restart the kernel to use updated packages.


### Retrieve data

Define the variables and levels we want to retrieve. Single-level variables ignore the "levels" parameter. Also note that not all variables in the ERA5 dataset are coded with their parameter names as of now. We also take a reduced sample of years in the dataset.

In [1]:
variables = ['geopotential']
levels = [500]
years = list(range(2013, 2019))

Initialize the data retriever. You'll want to change the directory to where you want to save the files.

In [2]:
import os
os.chdir(os.pardir)
from DLWP.data import ERA5Reanalysis

data_directory = '/usr/local/google/ilopezgp/ERA5_data_dlwp'
os.makedirs(data_directory, exist_ok=True)
era = ERA5Reanalysis(root_directory=data_directory, file_id='tutorial')
era.set_variables(variables)
era.set_levels(levels)

Download data! Automatically uses multi-processing to retrieve multiple files at a time. Note the parameter `hourly` says we're retrieving only every 3rd hour in the data, which is available hourly. The optional parameter passed to the retrieval package specifies that we want data interpolated to a 2-by-2 latitude-longitude grid.

In [3]:
era.retrieve(variables, levels, years=years, hourly=3,
             request_kwargs={'grid': [2., 2.]}, verbose=True, delete_temporary=True)

ERA5Reanalysis.retrieve: WARNING: file /usr/local/google/ilopezgp/ERA5_data_dlwp/tutorial_geopotential_500.nc already exists; omitting


Check that we got what we wanted after the retrieval is done:

In [4]:
era.open()
print(era.Dataset)

<xarray.Dataset>
Dimensions:    (latitude: 91, level: 1, longitude: 180, time: 17528)
Coordinates:
  * longitude  (longitude) float32 0.0 2.0 4.0 6.0 ... 352.0 354.0 356.0 358.0
  * latitude   (latitude) float32 90.0 88.0 86.0 84.0 ... -86.0 -88.0 -90.0
  * time       (time) datetime64[ns] 2013-01-01 ... 2018-12-31T21:00:00
  * level      (level) float32 500.0
Data variables:
    z          (time, level, latitude, longitude) float32 dask.array<chunksize=(17528, 1, 91, 180), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2021-07-02 18:23:46 GMT by grib_to_netcdf-2.20.0: /opt/ecmw...


### Process data for ingestion into DLWP

Now we use the DLWP.model.Preprocessor tool to generate a new data file ready for use in a DLWP Keras model. Some preliminaries... Note that we assign level "0" to the single-level 2m temperature data. I highly recommend using "pairwise" data processing, which means that each variable is matched to a level pair-wise. The length of the variables and levels lists should be the same. Also note that you only need to specify whole days in the dates. It takes care of the hourly data automatically.

In [6]:
import pandas as pd
from DLWP.data.era5 import get_short_name

dates = list(pd.date_range('2013-01-01', '2018-12-31', freq='D').to_pydatetime())
variables = ['z'] #get_short_name(variables)
levels = [500]
processed_file = '%s/tutorial_z500.nc' % data_directory

Process data! For proper use of data in a neural network, variables must be normalized relative to each other. This is typically done simply by removing mean and dividing by standard deviation (`scale_variables` option). To save on memory use, we normally calculate the global mean and std of the data in batches. Since this is a small dataset, we can use a large batch size to make it go faster.

In [7]:
from DLWP.model import Preprocessor

pp = Preprocessor(era, predictor_file=processed_file)
pp.data_to_series(batch_samples=10000, variables=variables, levels=levels, pairwise=True,
                  scale_variables=True, overwrite=True, verbose=True)

Preprocessor.data_to_samples: opening and formatting raw data
Preprocessor.data_to_samples: creating output file /usr/local/google/ilopezgp/ERA5_data_dlwp/tutorial_z500.nc
Preprocessor.data_to_samples: variable/level pair 1 of 1 (z/500)
Preprocessor.data_to_samples: calculating mean and std
Preprocessor.data_to_samples: writing batch 1 of 2
Preprocessor.data_to_samples: writing batch 2 of 2


Show our dataset, then clean up. We also save to a version with no string coordinates (might be needed for tempest-remap in the next tutorial).

In [8]:
print(pp.data)
pp.data.drop('varlev').to_netcdf(processed_file + '.nocoord')
era.close()
pp.close()

<xarray.Dataset>
Dimensions:     (lat: 91, lon: 180, sample: 17528, varlev: 1)
Coordinates:
  * lat         (lat) float32 90.0 88.0 86.0 84.0 ... -84.0 -86.0 -88.0 -90.0
  * lon         (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * varlev      (varlev) object 'z/500'
  * sample      (sample) datetime64[ns] 2013-01-01 ... 2018-12-31T21:00:00
Data variables:
    predictors  (sample, varlev, lat, lon) float32 dask.array<chunksize=(1, 1, 91, 180), meta=np.ndarray>
    mean        (varlev) float32 dask.array<chunksize=(1,), meta=np.ndarray>
    std         (varlev) float32 dask.array<chunksize=(1,), meta=np.ndarray>
Attributes:
    description:  Training data for DLWP
    scaling:      True
